In [ ]:
!curl -X POST \
    -H 'Content-Type':'aplication/json' \
    -d '{"data": [[45]]}' \
     http://e48fac8d-c927-4abc-9f58-e484631ca446.westus.azurecontainer.io/score

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import os, time, warnings, sys
import joblib
import numpy as np
import pandas as pd
from pylab import rcParams

warnings.filterwarnings(action="ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

os.getcwd()
os.listdir()

sal = pd.read_csv('survey_results_public.csv')
sal = sal.iloc[:,[3,5,6, 58,59]]

sal.shape

sal.CompTotal.describe(percentiles=[.8, .9, .95])

sal.CompTotal = sal.CompTotal.fillna(sal.CompTotal.median())
sal.Age = sal.Age.fillna(sal.Age.median())
sal.WorkWeekHrs = sal.WorkWeekHrs.fillna(sal.WorkWeekHrs.median())

sal = sal.loc[sal.CompTotal < 90_000, :]
# sal[["CompTotal", "Age"]] = sal[["CompTotal", "Age"]].fillna(method="median") 

sal.shape

X = sal.Age.values.reshape(-1, 1)
# X = sal.WorkWeekHrs.values.reshape(-1, 1)
y = sal.CompTotal.values

from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

lm = LinearRegression()
lm.fit(X_train, y_train)

print(f"Intercept: {lm.intercept_}")
print(f"Slope: {lm.coef_[0]}")

from sklearn.metrics import mean_squared_error
y_predict = lm.predict(X_test)
mse = mean_squared_error(y_predict, y_test)
mse
print(f"MSE: {round(mse,2)}")

filename = "sal_model.pkl"
joblib.dump(lm, filename)

Intercept: 53587.606708923035
Slope: -98.69396755367482
MSE: 507742324.54


['sal_model.pkl']

In [2]:
os.getcwd()

'C:\\Users\\barth\\Data\\DataBases\\Tutorial_Azure_ML'

In [ ]:
#! flask/bin/python

In [ ]:
from flask import Flask, jsonify
import joblib

filename = "sal_model.pkl"
app = Flask(__name__)

@app.route('/')
def index():
    return "Stackoverflow Salary Predictor"

@app.route('/sal/<int:x>', methods=['GET'])
def predict(x):
    loaded_model.predict([[x]])[0]
    sal = jsonify({'salary': round(y,2)})
    return sal

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80)